## Using ReAct Framework to analyse a stock.
 - Get the latest stock data from a data source.
 - Get the stock symbol, its financials
 - Get the news data headlines from the internet.

* We will use Llama3.1 to do this.

In [2]:
import json
import sys
import os
from typing import Optional

from llama_index.llms.ollama import Ollama
from llama_index.llms.groq import Groq
from llama_index.core import Settings
# from llama_index.tools.google import GoogleSearchToolSpec
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core import PromptTemplate

from dotenv import load_dotenv

# finance tool
import yfinance as yf
import datetime as dt

# get the prompts
sys.path.append(os.path.dirname(os.path.abspath("../stock_analyser_react.ipynb")))
from prompts.react_agent_prompts import LLAMAINDEX_REACT_BASE_PROMPT, STOCK_ANALYSER_PROMPT


# Load the env variables
load_dotenv("../secrets/local.env")

True

In [3]:
# LLM ollama
# llm = Ollama(model='llama3.1:latest', request_timeout=120.0, temperature=0.01)
# Settings.llm = llm

# LLM Groq
llm = Groq(model="llama-3.1-70b-versatile")
Settings.llm = llm

In [4]:
stream_response = llm.stream_complete("What is the stock symbol for Bajaj Finance?")

for t in stream_response:
    print(t.delta, end="")


# Note: how the model is hallucinating

The stock symbol for Bajaj Finance is BAJFINANCE.

In [5]:
# define the tools

    
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.
    
    Returns:
        int: The sum of a and b
    """
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer
    
    Args:
        a (int): The first integer.
        b (int): The second integer.
    
    Returns:
        int: The sum of a and b
    """
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The quotient of a divided by b.
    """
    return a // b


# yahoo finance financials fetcher
def get_financial_statements(ticker: str) -> str:
    """Fetches the financial statements data from yahoo finance.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.

    Returns:
        str: A string containing the financial statement data in a formatted way.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[-1] > 3:
        balance_sheet = balance_sheet.iloc[:, :3]

    balance_sheet = balance_sheet.dropna(how="any")
    balance_sheet = "\n" + balance_sheet.to_string()

    return balance_sheet


# get stock info and recommendations summary
def get_stockinfo(ticker: str) -> str:
    """Provides the detailed financial and general information about the stock ticker.
        Also provides a table for analyst recommendations for the ticker.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.
    
    Returns:
        str: A string containing the stock info and a summary of recommendations.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"
    company = yf.Ticker(ticker)

    stock_info = company.info
    try:
        recommendations_summary = company.get_recommendations()
    except:
        recommendations_summary = ""

    # TODO: add units and convert to easily understandable units

    include_info = ["industry", "sector", "longBuisnessSummary", "previousClose", "dividendRate", "dividentYield", "beta", "forwardPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "currency", "bookValue", "priceToBook", "earningsQuarterlyGrowth", "trailingEps", "forwardEps", "52WeekChange", "totalCashPerShare", "ebidta", "totabDebt", "totalCashPerShare", "debtToEquity", "revenuePerShare", "earningsGrowth", "revenueGrowth", "grossMargins", "ebidtaMargins", "operatingMargins", ]
    response = "#### Stock info:\n"

    for key, val in stock_info.items():
        if key in include_info:
            response += f"{key}: {val}\n"

    response += "#### Analyst Recommendations:\n"
    response += f"\n{recommendations_summary}"

    return response


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
sub_tool = FunctionTool.from_defaults(fn=subtract)
div_tool = FunctionTool.from_defaults(fn=divide)

# financial sheets
fin_statements = FunctionTool.from_defaults(fn=get_financial_statements)

# stock information and the analyst recommendations
stockinfo = FunctionTool.from_defaults(fn=get_stockinfo)

ddg_tools = DuckDuckGoSearchToolSpec().to_tool_list()
ddg_full_search_tool = ddg_tools[1]
ddg_instant_search_tool = ddg_tools[0]

In [6]:
multiply.__annotations__

{'a': int, 'b': int, 'return': int}

In [5]:
ddg_instant_search_tool.metadata

ToolMetadata(description='duckduckgo_instant_search(query: str) -> List[Dict]\n\n        Make a query to DuckDuckGo api to receive an instant answer.\n\n        Args:\n            query (str): The query to be passed to DuckDuckGo.\n        ', name='duckduckgo_instant_search', fn_schema=<class 'llama_index.core.tools.utils.duckduckgo_instant_search'>, return_direct=False)

In [7]:
# Define a ReactAgent
agent = ReActAgent.from_tools(tools=[ddg_full_search_tool, ddg_instant_search_tool, fin_statements, stockinfo],
                   llm=llm,
                   verbose=True,
                   max_iterations=25,
                   context="You are a top Research Analyst with a Phd in Economics. You analyse equity markets very well and are proficient in creating detailed analysis reports. You do not provide recommendations only analyse company stocks.ƒ")


# have a look at the agent prompt
agent.get_prompts()

{'agent_worker:system_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['tool_desc', 'context', 'tool_names'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n{tool_desc}\n\nHere is some context to help you answer the question and plan:\n{context}\n\n\n## Output Format\n\nPlease answer in the same language as the question and use the following format:\n\n```\nThought: The current language of the user is: (user\'s language). I need to use a tool to help me answer the 

In [8]:
# assign the prompt to the agent
stock_analyser_prompt = PromptTemplate(template=STOCK_ANALYSER_PROMPT)
agent.update_prompts(
    {'agent_worker:system_prompt': stock_analyser_prompt}
)

In [10]:
response = agent.chat("Compare Bajaj Finance and Bajaj Finserv")

> Running step 87fe1a3b-97e6-4c6e-9f26-8868ef17e527. Step input: Compare Bajaj Finance and Bajaj Finserv


Thought: The current language of the user is: English. I need to extract the company names from the input query.
Action: Extract
Action Input: {'companies': ['Bajaj Finance', 'Bajaj Finserv']}
Observation: Error: No such tool named `Extract`.
> Running step c1ed79c4-a14e-4d9d-99c2-fd8d717027d1. Step input: None
Thought: The current language of the user is: English. I need to extract the company names from the input query. Since there is no tool for extraction, I will manually extract the company names and proceed with the analysis.

The company names are Bajaj Finance and Bajaj Finserv.

Next, I need to get the company symbols for Bajaj Finance and Bajaj Finserv by using the "duckduckgo_full_search" tool.
Action: duckduckgo_full_search
Action Input: {'query': 'Bajaj Finance NSE/BSE ticker', 'region': 'in', 'max_results': 1}
Observation: [{'title': 'Bajaj Finance Limited Share Price Today, Stock Price, Live NSE News ...', 'href': 'https://www.nseindia.com/get-quotes/equity?symbol=BAJFIN

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BAJFINANCE.NS?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=BAJFINANCE.NS&crumb=ZjbBJqK%2FSM8


Observation: #### Stock info:
industry: Credit Services
sector: Financial Services
previousClose: 7345.75
dividendRate: 36.0
beta: 1.017
volume: 1567645
marketCap: 4555488100352
fiftyTwoWeekLow: 6187.8
fiftyTwoWeekHigh: 8192.0
currency: INR
bookValue: 1241.009
priceToBook: 5.9350896
earningsQuarterlyGrowth: 0.138
trailingEps: 242.41
52WeekChange: -0.026878953
totalCashPerShare: 145.924
debtToEquity: 305.489
revenuePerShare: 537.325
earningsGrowth: 0.115
revenueGrowth: 0.18
grossMargins: 0.93878996
operatingMargins: 0.60258
#### Analyst Recommendations:

Empty DataFrame
Columns: []
Index: []
> Running step e9f4c8d2-86da-4d70-99a6-ed91dfb5c1a7. Step input: None
Thought: I have obtained the stock information and recommendations for Bajaj Finance.

Next, I need to get the stock information and recommendations for Bajaj Finserv by using the "get_stockinfo_recommendations" tool.
Action: get_stockinfo_recommendations
Action Input: {'ticker': 'BAJAJFINSV'}
Observation: #### Stock info:
industr

In [11]:
response

AgentChatResponse(response="Bajaj Finance and Bajaj Finserv are both strong financial services companies with their unique strengths and weaknesses. Bajaj Finance has a stronger financial position and higher growth potential, while Bajaj Finserv has a more attractive valuation and lower debt levels. I am just an AI Research analyst assistant. I am not a financial advisor. I don't provide investment advice or recommendations.", sources=[ToolOutput(content='Error: No such tool named `Extract`.', tool_name='Extract', raw_input={'kwargs': {'companies': ['Bajaj Finance', 'Bajaj Finserv']}}, raw_output='Error: No such tool named `Extract`.', is_error=True), ToolOutput(content="[{'title': 'Bajaj Finance Limited Share Price Today, Stock Price, Live NSE News ...', 'href': 'https://www.nseindia.com/get-quotes/equity?symbol=BAJFINANCE', 'body': 'Bajaj Finance Limited Share Price Today, Live NSE Stock Price: Get the latest Bajaj Finance Limited news, company updates, quotes, offers, annual financi